In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/10/06 10:17:32 WARN Utils: Your hostname, adi-laptop resolves to a loopback address: 127.0.1.1; using 172.23.90.35 instead (on interface eth0)
22/10/06 10:17:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 10:17:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)
sdf_consumer.show(5, truncate=False)

+-----------------+-----------------------------+-----+--------+------+-----------+
|name             |address                      |state|postcode|gender|consumer_id|
+-----------------+-----------------------------+-----+--------+------+-----------+
|Yolanda Williams |413 Haney Gardens Apt. 742   |WA   |6935    |Female|1195503    |
|Mary Smith       |3764 Amber Oval              |NSW  |2782    |Female|179208     |
|Jill Jones MD    |40693 Henry Greens           |NT   |862     |Female|1194530    |
|Lindsay Jimenez  |00653 Davenport Crossroad    |NSW  |2780    |Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA   |6355    |Female|712975     |
+-----------------+-----------------------------+-----+--------+------+-----------+
only showing top 5 rows



Convert postcodes to strings with leading zeros

In [4]:
sdf_consumer = sdf_consumer.withColumn('postcode', F.format_string("%04d", F.col('postcode').cast('int')))

In [5]:
sdf_consumer.count()

499999

In [6]:
VIC_sdf = sdf_consumer.where(
    (F.col('state') == 'VIC' ))

NSW_sdf = sdf_consumer.where(
    (F.col('state') == 'NSW' ))

QLD_sdf = sdf_consumer.where(
    (F.col('state') == 'QLD' ))

WA_sdf = sdf_consumer.where(
    (F.col('state') == 'WA' ))

TAS_sdf = sdf_consumer.where(
    (F.col('state') == 'TAS' ))

SA_sdf = sdf_consumer.where(
    (F.col('state') == 'SA' ))

ACT_sdf = sdf_consumer.where(
    (F.col('state') == 'ACT' ))

NT_sdf = sdf_consumer.where(
    (F.col('state') == 'NT' ))

In [ ]:
VIC_sdf = VIC_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >= '3000') & (F.col('postcode') <= '3999')
    | ((F.col('postcode') >=  '8000') & (F.col('postcode') <= '8999')), F.col('postcode'))
)
NT_sdf = NT_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >= '0800') & (F.col('postcode') <= '0999'), F.col('postcode'))
)
TAS_sdf = TAS_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >=  '7000') & (F.col('postcode') <= '7999'), F.col('postcode'))
)
WA_sdf = WA_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >=  '6000') & (F.col('postcode') <= '6797')
    | (F.col('postcode') >=  '6800') & (F.col('postcode') <= '6999'), F.col('postcode'))
)
SA_sdf = SA_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >=  '5000') & (F.col('postcode') <= '5999'), F.col('postcode'))
)
QLD_sdf = QLD_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >=  '4000') & (F.col('postcode') <= '4999')
    | (F.col('postcode') >=  '9000') & (F.col('postcode') <= '9999'), F.col('postcode'))
)
NSW_sdf = NSW_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >=  '1000') & (F.col('postcode') <= '2599')
    | (F.col('postcode') >=  '2619') & (F.col('postcode') <= '2899')
    | (F.col('postcode') >=  '2921') & (F.col('postcode') <= '2999'), F.col('postcode'))
)
ACT_sdf = ACT_sdf.withColumn(
    'postcode',
    F.when((F.col('postcode') >=  '0200') & (F.col('postcode') <= '0300')
    | (F.col('postcode') >=  '2600') & (F.col('postcode') <= '2618')
    | (F.col('postcode') >=  '2900') & (F.col('postcode') <= '2920'), F.col('postcode'))
)

In [17]:
cleaned_consumer = VIC_sdf.union(NSW_sdf) \
        .union(QLD_sdf) \
        .union(WA_sdf) \
        .union(TAS_sdf) \
        .union(SA_sdf) \
        .union(ACT_sdf) \
        .union(NT_sdf)

cleaned_consumer.count()

499999

In [22]:
cleaned_consumer.where(F.isnull(F.col('postcode'))).count()

1209

1209 (0.24%) instances had postcodes that did not match their state.

In [23]:
cleaned_consumer.write.mode('overwrite').parquet('../data/curated/cleaned_consumers.parquet')

In [24]:
cleaned_consumer.show(5)

+----------------+--------------------+-----+--------+-----------+-----------+
|            name|             address|state|postcode|     gender|consumer_id|
+----------------+--------------------+-----+--------+-----------+-----------+
|Jillian Gonzales|461 Ryan Common S...|  VIC|    3220|     Female|    1058499|
|    Eugene Lucas|33983 Kevin Drive...|  VIC|    3063|Undisclosed|     428325|
|    Lance Butler|   8943 Kenneth Camp|  VIC|    3332|       Male|    1343547|
|   Alyssa Wilson|  44353 Nathan Ridge|  VIC|    3719|     Female|    1331093|
|    James Norris|   790 Ramos Landing|  VIC|    3234|Undisclosed|    1390367|
+----------------+--------------------+-----+--------+-----------+-----------+
only showing top 5 rows



In [27]:
male_count = cleaned_consumer.where(F.col('gender') == 'Male').count()
female_count = cleaned_consumer.where(F.col('gender') == 'Female').count()
undisclosed_count = cleaned_consumer.where(F.col('gender') == 'Undisclosed').count()

In [31]:
import pandas as pd

In [37]:
data = {'gender': ['male', 'female', 'undisclosed'], 'count': [male_count, female_count, undisclosed_count]}
gender_counts = pd.DataFrame(data)

In [38]:
gender_counts

,gender,count
0,male,224979
1,female,224946
2,undisclosed,50074


In [39]:
import plotly.express as px

In [42]:
px.bar(gender_counts, x='gender', y='count', color='gender')

In [43]:
state_counts = cleaned_consumer.groupBy('state').count()

In [45]:
state_counts = state_counts.toPandas()

In [47]:
px.bar(state_counts, x='state', y='count', color='state')